# Module 7 - Programming Assignment

## Directions

There are general instructions on Blackboard and in the Syllabus for Programming Assignments. This Notebook also has instructions specific to this assignment. Read all the instructions carefully and make sure you understand them. Please ask questions on the discussion boards or email me at `EN605.445@gmail.com` if you do not understand something.

<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
You must follow the directions *exactly* or you will get a 0 on the assignment.
</div>

You must submit a zip file of your assignment and associated files (if there are any) to Blackboard. The zip file will be named after you JHED ID: `<jhed_id>.zip`. It will not include any other information. Inside this zip file should be the following directory structure:

```
<jhed_id>
    |
    +--module-01-programming.ipynb
    +--module-01-programming.html
    +--(any other files)
```

For example, do not name  your directory `programming_assignment_01` and do not name your directory `smith122_pr1` or any else. It must be only your JHED ID.

Imports here if needed.

In [1]:
import copy

# Forward Planner

## Unify

Use the accompanying `unification.py` file for unification. For this assignment, you're almost certainly going to want to be able to:

1. specify the problem in terms of S-expressions.
2. parse them.
3. work with the parsed versions.

`parse` and `unification` work exactly like the programming assignment for last time.

In [2]:
from unification import parse, unification

## Forward Planner

In this assigment, you're going to implement a Forward Planner. What does that mean? If you look in your book, you will not find pseudocode for a forward planner. It just says "use state space search" but this is less than helpful and it's a bit more complicated than that. **(but please please do not try to implement STRIPS or GraphPlan...that is wrong).**

At a high level, a forward planner takes the current state of the world $S_0$ and attempts to derive a plan, basically by Depth First Search. We have all the ingredients we said we would need in Module 1: states, actions, a transition function and a goal test. We have a set of predicates that describe a state (and therefore all possible states), we have actions and we have, at least, an implicit transition function: applying an action in a state causes the state to change as described by the add and delete lists.

Let's say we have a drill that's an item, two places such as home and store, and we know that I'm at home and the drill is at the store and I want to go buy a drill (have it be at home). We might represent that as:

<code>
start_state = [
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Store)"
]
</code>

And we have a goal state:

<code>
goal = [
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)"
]
</code>

The actions/operators are:

<code>
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}
</code>

These will all need to be parsed from s-expressions to the underlying Python representation before you can use them. You might as well do it at the start of your algorithm, once. The order of the conditions is *not* arbitrary. It is much, much better for the unification and backtracking if you have the "type" predicates (item, place, agent) before the more complex ones. Trust me on this.

As for the algorithm itself, there is going to be an *outer* level of search and an *inner* level of search.

The *outer* level of search that is exactly what I describe here: you have a state, you generate successor states by applying actions to the current state, you examine those successor states as we did at the first week of the semester and if one is the goal you stop, if you see a repeat state, you put it on the explored list (you should implement graph search not tree search). What could be simpler?

It turns out the Devil is in the details. There is an *inner* level of search hidden in "you generate successor states by applying actions to the current state". Where?

How do you know if an action applies in a state? Only if the preconditions successfully unify with the current state. That seems easy enough...you check each predicate in the conditions to see if it unifies with the current state and if it does, you use the substitution list on the action, the add and delete lists and create the successor state based on them.

Except for one small problem...there may be more than one way to unify an action with the current state. You must essentially search for all successful unifications of the candidate action and the current state. This is where my question through the semester appliesm, "how would you modify state space search to return all the paths to the goal?"

Unification can be seen as state space search by trying to unify the first precondition with the current state, progressively working your way through the precondition list. If you fail at any point, you may need to backtrack because there might have been another unification of that predicate that would succeed. Similarly, as already mentioned, there may be more than one.

So...by using unification and a properly defined <code>successors</code> function, you should be able to apply graph based search to the problem and return a "path" through the states from the initial state to the goal. You'll definitely want to use graph-based search since <code>( drive Me Store), (drive Me Home), (drive Me Store), (drive Me Home), (drive Me Store), (buy Me Store Drill), (drive Me Home)</code> is a valid plan.

Your function should return the plan...but if you pass an extra debug=True parameter, it should also return the intermediate *states* as well as the actions.

-----

**pretty_print**

Print the plan line-by-line.

In [3]:
def pretty_print( plan):
    for line in plan:
        print line

**parse_collection**

This function takes in any kind of S-expression, whether in string, dictionary, or list form (or nested dictionary/list) and modifies it in-place to python syntax. If the given collection is a list, it iterates through the list and recursively calls itself on the elements of the list. If it is a dictionary, it recursively calls itself on each value and stores the result in the key. If it is a string (the base case), it is sent to the parse function to be converted.

In [4]:
def parse_collection( coll):
    if type( coll) == list:
        for index in range(len( coll)):
            coll[index] = parse_collection( coll[index])
    elif type( coll) == dict:
        for key in coll:
            coll[key] = parse_collection( coll[key])
    elif type( coll) == str:
        coll = parse( coll)
    return coll

**initialize**

Creates the data structures used by the forward planner. The start, goal, and actions structures are converted to python syntax (copies are created to leave the originals intact). The plan is initialized (just the actions if no DEBUG flag, otherwise the states are included too and initialized to be the starting state). Finally, the frontier (stack) is initialized to a tuple containing the starting state and initial plan. All of the members are returned.

In [5]:
def initialize( start_state, goal_state, actions, DEBUG):
    start = parse_collection( copy.deepcopy( start_state))
    goal = parse_collection( copy.deepcopy( goal_state))
    moves = parse_collection( copy.deepcopy( actions))
    if DEBUG:
        plan = [start]
    else:
        plan = []
    return start, goal, moves, [], [(start, plan)]

**substitute**

This function takes an action dictionary and substitution list dictionary and performs any necessary substitutions. It replaces any instance of the keys from the substitution list with their corresponding values in any of the members of the action dictionary. The original is left intact, and a new, modified copy is returned.

In [6]:
def substitute( action, substitution_list):
    new_action = copy.deepcopy(action)
    for key in substitution_list:
        new_action["action"] = [x.replace( key, substitution_list[key]) for x in new_action["action"]]
        for index in range( len( new_action["add"])):
            new_action["add"][index] = [x.replace( key, substitution_list[key]) for x in new_action["add"][index]]
        for index in range( len( new_action["delete"])):
            new_action["delete"][index] = [x.replace( key, substitution_list[key]) for x in new_action["delete"][index]]
        for index in range( len( new_action["conditions"])):
            new_action["conditions"][index] = [x.replace( key, substitution_list[key]) for x in new_action["conditions"][index]]
    return new_action

**is_valid_action**

Determines whether an action's prerequisites are satisfied by the given state. Every prerequisite must be satisfied for an action to be valid. A condition is considered valid if there exists at least one predicate in the state that successfully unifies with the condition. If a condition is not satisfied, the function immediately returns false. True is returned when the final condition is determined to be satisfied.

In [7]:
def is_valid_action( state, action):
    for condition in action["conditions"]:
        condition_satisfied = False
        for predicate in state:
            if unification( predicate, condition) != False:
                condition_satisfied = True
                break
        if condition_satisfied == False:
            return False
    return True

**exists**

Helper function to determine whether or not an action already exists in the given list. It does this by comparing the members after lexigraphically sorting them, to avoid cases where the lists contain the same expressions in different order.

In [8]:
def exists( action, container):
    for existing in container:
        if sorted(action["action"]) == sorted(existing["action"]):
            return True
    return False

**valid_actions**

This function is the inner search for forward planning. The goal of this function is to return a list of all valid actions that can be created from the current state and action. It does this using depth-first search. The stack is initialized to the starting action, and an empty explored list is created. While the stack has data, the top action is popped off. Every condition is unified with every predicate in the state. For any that successfully unify, the substitution list is applied to the current action and the new action is pushed onto the stack (if it's new). This does not affect the current action, as it gets unified with every predicate without being modified.

If every condition is compared to every predicate in the state and yields an empty substitution list each time, we know all of the variables have been substituted. At this point, it is checked for compatibility with the state. If it works with the current state, it is a possible action and is stored off. At the end of the function, all possible valid actions based on all possible unifications are returned.

In [9]:
def valid_actions( state, action):
    all_possible_actions = []
    stack = [action]
    explored = []
    
    while len(stack) > 0:
        curr_action = stack.pop()
        unified = True
        for conditionIndex in range( len( curr_action["conditions"])):
            condition = curr_action["conditions"][conditionIndex]
            for pred in state:
                substitution_list = unification( pred, condition)
                if substitution_list != False:
                    if len( substitution_list) > 0:
                        unified = False
                    new_action = substitute( curr_action, substitution_list)
                    if not exists( new_action, stack) and not exists( new_action, explored):
                        stack.append( new_action)
        if unified:
            if not exists( curr_action, all_possible_actions):
                if is_valid_action( state, curr_action):
                    all_possible_actions.append( curr_action)
        explored.append( curr_action)
    return all_possible_actions

**next_state**

Applies an action to a state and returns the new state that is generated. A copy is made so as to not modify the old state. All states in the action's delete list are removed from the state's predicate list. All states in the action's add list are then added to the predicate list and the new state is returned.

In [10]:
def next_state( old_state, action):
    new_state = copy.deepcopy(old_state)
    for delete in action["delete"]:
        new_state.remove(delete)
    for add in action["add"]:
        new_state.append(add)
    return new_state

**successors**

This function returns a list of all successor states from the current state. It first gets a list of possible action instantiations from the current state using each action type. Each of these actions are applied to generate a new destination state. If the state changes (to avoid redundant actions from A to A), the action taken is added to a new copy of the plan and stored along with the successor. This list of state-plan tuples are returned.

In [11]:
def successors( state, actions, plan, DEBUG):
    successors = []
    for actionKey in actions:
        action = actions[actionKey]
        new_actions = valid_actions( state, action)
        for new_action in new_actions:
            new_state = next_state( state, new_action)
            if sorted(new_state) != sorted(state):
                new_plan = copy.deepcopy( plan)
                new_plan.append( new_action["action"])
                if DEBUG:
                    new_plan.append( new_state)
                successors.append( (new_state, new_plan))
    return successors

(you can just overwrite that one and add as many others as you need). Remember to follow the **Guidelines**.


-----

So you need to implement `forward_planner` as described above. `start_state`, `goal` and `actions` should all have the layout above and be s-expressions.

Your implementation should return the plan as a **List of instantiated actions**. If `debug=True`, you should print out the intermediate states of the plan as well.

In [12]:
def forward_planner( start_state, goal, actions, debug=False):
    start, end, moves, explored, stack = initialize( start_state, goal, actions, debug)
    while len( stack) > 0:
        curr_state, curr_plan = stack.pop()
        if sorted(curr_state) == sorted(end):
            return curr_plan
        for successor, curr_plan in successors( curr_state, moves, copy.deepcopy(curr_plan), debug):
            if successor not in explored and successor not in stack:
                stack.append( (successor, curr_plan))
        explored.append( curr_state)
    return []

You will be solving the problem from above. Here is the start state:

In [13]:
start_state = [
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Store)"
]

The goal state:

In [14]:
goal = [
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)"
]

and the actions/operators:

In [15]:
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}

In [16]:
plan = forward_planner( start_state, goal, actions)
pretty_print( plan)

['drive', 'Me', 'Home', 'Store']
['buy', 'Me', 'Store', 'Drill']
['drive', 'Me', 'Store', 'Home']


In [17]:
plan_with_states = forward_planner( start_state, goal, actions, debug=True)
pretty_print( plan_with_states)

[['item', 'Drill'], ['place', 'Home'], ['place', 'Store'], ['agent', 'Me'], ['at', 'Me', 'Home'], ['at', 'Drill', 'Store']]
['drive', 'Me', 'Home', 'Store']
[['item', 'Drill'], ['place', 'Home'], ['place', 'Store'], ['agent', 'Me'], ['at', 'Drill', 'Store'], ['at', 'Me', 'Store']]
['buy', 'Me', 'Store', 'Drill']
[['item', 'Drill'], ['place', 'Home'], ['place', 'Store'], ['agent', 'Me'], ['at', 'Me', 'Store'], ['at', 'Drill', 'Me']]
['drive', 'Me', 'Store', 'Home']
[['item', 'Drill'], ['place', 'Home'], ['place', 'Store'], ['agent', 'Me'], ['at', 'Drill', 'Me'], ['at', 'Me', 'Home']]
